Visão Geral do Projeto
Este projeto consiste em criar um assistente que:

Recebe solicitações de reuniões em linguagem natural.
Entende restrições e preferências dos participantes.
Verifica os calendários dos participantes.
Agenda reuniões de forma eficiente.
Envia convites automaticamente.
Utilizaremos as seguintes tecnologias:

Python: Linguagem de programação principal.
LangGraph: Para criar fluxos de trabalho baseados em estados.
LangChain: Para interação com modelos de linguagem natural (LLMs).
Bibliotecas de Calendário e Email: Para acessar calendários e enviar emails (exemplo: caldav, smtplib).

Parte 1: Preparação do Ambiente
1.1. Instalação de Bibliotecas Necessárias
Execute o seguinte comando em uma célula do Jupyter Notebook para instalar as bibliotecas necessárias:

In [2]:
!pip install langgraph langchain-openai langchain-core openai caldav python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 16.4 MB/s eta 0:00:0031m17.6 MB/s eta 0:00:01


In [3]:
# Importações necessárias
import os
import re
import json
import datetime
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, START, END
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Inicializar o modelo de linguagem 
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


Notas:

caldav: Biblioteca para acessar calendários via protocolo CalDAV.
python-dotenv: Para carregar variáveis de ambiente de um arquivo .env.
langgraph, langchain-openai, langchain-core, openai: Bibliotecas para interação com modelos de linguagem e construção do fluxo de trabalho.

Parte 2: Definição das Estruturas de Dados e Estado
2.1. Definição das Estruturas de Dados

In [4]:
# Estruturas de dados para participantes e reuniões
class Participant(TypedDict):
    name: str
    email: str
    calendar: List[Dict[str, any]]  # Lista de eventos no calendário

class MeetingRequest(TypedDict):
    requester: str
    attendees: List[str]
    preferred_times: List[str]
    duration: int  # em minutos
    subject: str
    constraints: Dict[str, any]

class ScheduledMeeting(TypedDict):
    subject: str
    time: datetime.datetime
    duration: int
    attendees: List[str]
    organizer: str


2.2. Definição do Estado do Fluxo de Trabalho



In [5]:
# Definição do estado
class State(TypedDict):
    meeting_request: MeetingRequest
    participants: Dict[str, Participant]
    scheduled_meeting: ScheduledMeeting
    errors: List[str]


Parte 3: Implementação das Funções de Fluxo de Trabalho
3.1. Função para Processar a Solicitação de Reunião

In [6]:
def parse_meeting_request(state: State) -> State:
    """
    Analisa a solicitação de reunião em linguagem natural e extrai informações relevantes.
    """
    meeting_request_text = state['meeting_request']['raw_text']
    
    prompt = ChatPromptTemplate.from_template(
        """
Você é um assistente que extrai informações de solicitações de reuniões.
A partir do texto abaixo, extraia:
- Solicitante (nome)
- Lista de participantes (nomes)
- Horários preferenciais (lista de datas e horários)
- Duração da reunião em minutos
- Assunto da reunião
- Restrições adicionais ou preferências

Retorne as informações em formato JSON, sem texto adicional.

Exemplo de formato:
{{
  "requester": "Nome do Solicitante",
  "attendees": ["Participante 1", "Participante 2"],
  "preferred_times": ["2023-10-01 14:00", "2023-10-02 10:00"],
  "duration": 60,
  "subject": "Assunto da Reunião",
  "constraints": {{}}
}}

Texto da solicitação:
{meeting_request_text}
        """
    )
    response = llm.invoke(prompt.format(meeting_request_text=meeting_request_text))
    try:
        meeting_info = json.loads(response.content)
        state['meeting_request'].update(meeting_info)
    except json.JSONDecodeError as e:
        state['errors'].append(f"Erro ao analisar a solicitação: {e}")
    return state


3.2. Função para Verificar Disponibilidade dos Participantes



In [7]:
def check_availability(state: State) -> State:
    """
    Verifica a disponibilidade dos participantes nos horários preferenciais.
    """
    available_times = []
    for time_str in state['meeting_request']['preferred_times']:
        time_obj = datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M")
        duration = datetime.timedelta(minutes=state['meeting_request']['duration'])
        end_time = time_obj + duration
        
        # Verificar disponibilidade de todos os participantes
        all_available = True
        for attendee in state['meeting_request']['attendees']:
            calendar = state['participants'][attendee]['calendar']
            for event in calendar:
                event_start = event['start_time']
                event_end = event['end_time']
                # Se houver conflito, marcar como indisponível
                if (time_obj < event_end) and (end_time > event_start):
                    all_available = False
                    break
            if not all_available:
                break
        if all_available:
            available_times.append(time_obj)
    
    if available_times:
        # Agendar no primeiro horário disponível
        state['scheduled_meeting'] = {
            "subject": state['meeting_request']['subject'],
            "time": available_times[0],
            "duration": state['meeting_request']['duration'],
            "attendees": state['meeting_request']['attendees'],
            "organizer": state['meeting_request']['requester']
        }
    else:
        state['errors'].append("Não foi possível encontrar um horário disponível.")
    return state


3.3. Função para Enviar Convites

In [8]:
def send_invitations(state: State) -> State:
    """
    Envia convites para os participantes.
    """
    if 'scheduled_meeting' not in state:
        state['errors'].append("Nenhuma reunião agendada para enviar convites.")
        return state
    
    meeting = state['scheduled_meeting']
    for attendee in meeting['attendees']:
        email = state['participants'][attendee]['email']
        # Aqui você implementaria o envio de email real
        print(f"Enviando convite para {attendee} ({email}) para a reunião '{meeting['subject']}' em {meeting['time']}.")
    return state


Parte 4: Configuração do Fluxo de Trabalho com LangGraph
4.1. Construção do Grafo de Estados

In [9]:
# Inicializar o grafo de estado
workflow = StateGraph(State)

# Adicionar nós
workflow.add_node("parse_meeting_request", parse_meeting_request)
workflow.add_node("check_availability", check_availability)
workflow.add_node("send_invitations", send_invitations)

# Definir as arestas
workflow.add_edge(START, "parse_meeting_request")
workflow.add_edge("parse_meeting_request", "check_availability")
workflow.add_edge("check_availability", "send_invitations")
workflow.add_edge("send_invitations", END)

# Definir o ponto de entrada
workflow.set_entry_point("parse_meeting_request")

# Compilar o grafo
app = workflow.compile()


4.2. Função Principal para Executar o Fluxo de Trabalho



In [10]:
def schedule_meeting(meeting_request_text: str, participants: Dict[str, Participant]):
    """
    Processa a solicitação de reunião e agenda se possível.
    
    Args:
        meeting_request_text (str): Texto da solicitação em linguagem natural.
        participants (Dict[str, Participant]): Informações dos participantes.
    
    Returns:
        State: Estado final após o processamento.
    """
    initial_state = State(
        meeting_request={"raw_text": meeting_request_text},
        participants=participants,
        scheduled_meeting={},
        errors=[]
    )
    final_state = app.invoke(initial_state)
    return final_state


Parte 5: Exemplo de Uso

5.1. Dados de Exemplo dos Participantes

In [11]:
# Dados de exemplo dos participantes
participants = {
    "Alice": {
        "name": "Alice",
        "email": "alice@example.com",
        "calendar": [
            {"start_time": datetime.datetime(2023, 10, 1, 9, 0), "end_time": datetime.datetime(2023, 10, 1, 10, 0)},
            {"start_time": datetime.datetime(2023, 10, 1, 14, 0), "end_time": datetime.datetime(2023, 10, 1, 15, 0)},
        ]
    },
    "Bob": {
        "name": "Bob",
        "email": "bob@example.com",
        "calendar": [
            {"start_time": datetime.datetime(2023, 10, 1, 11, 0), "end_time": datetime.datetime(2023, 10, 1, 12, 0)},
            {"start_time": datetime.datetime(2023, 10, 1, 16, 0), "end_time": datetime.datetime(2023, 10, 1, 17, 0)},
        ]
    },
    "Carlos": {
        "name": "Carlos",
        "email": "carlos@example.com",
        "calendar": [
            {"start_time": datetime.datetime(2023, 10, 1, 13, 0), "end_time": datetime.datetime(2023, 10, 1, 14, 0)},
            {"start_time": datetime.datetime(2023, 10, 1, 15, 0), "end_time": datetime.datetime(2023, 10, 1, 16, 0)},
        ]
    },
}


5.2. Solicitação de Reunião em Linguagem Natural



In [12]:
# Solicitação de reunião em linguagem natural
meeting_request_text = """
Olá,

Gostaria de agendar uma reunião sobre o projeto X. Seria ideal se pudéssemos nos reunir amanhã às 10h ou às 15h. A reunião deve durar cerca de 60 minutos.

Participantes: Alice, Bob e Carlos.

Atenciosamente,
Maria
"""


5.3. Execução do Fluxo de Trabalho


In [14]:
# Executar o agendamento
final_state = schedule_meeting(meeting_request_text, participants)


Enviando convite para Alice (alice@example.com) para a reunião 'Projeto X' em 2023-10-01 10:00:00.
Enviando convite para Bob (bob@example.com) para a reunião 'Projeto X' em 2023-10-01 10:00:00.
Enviando convite para Carlos (carlos@example.com) para a reunião 'Projeto X' em 2023-10-01 10:00:00.


5.4. Verificar o Resultado

In [15]:
# Verificar se houve erros
if final_state['errors']:
    print("Erros encontrados:")
    for error in final_state['errors']:
        print(f"- {error}")
else:
    meeting = final_state['scheduled_meeting']
    print(f"Reunião agendada com sucesso!\nAssunto: {meeting['subject']}\nHorário: {meeting['time']}\nParticipantes: {', '.join(meeting['attendees'])}")


Reunião agendada com sucesso!
Assunto: Projeto X
Horário: 2023-10-01 10:00:00
Participantes: Alice, Bob, Carlos


Notas Finais
Interação com Calendários Reais: Para tornar o script funcional em um ambiente real, você precisaria integrar com APIs de calendário (como Google Calendar API) e implementar autenticação adequada.

Envio de Emails Reais: Para enviar emails, você pode usar a biblioteca smtplib do Python e configurar um servidor SMTP.

Melhorias Possíveis:

Adicionar tratamento de fusos horários.
Implementar lógica para encontrar horários alternativos se nenhum dos preferidos estiver disponível.
Salvar os eventos nos calendários dos participantes.